# Mnist classification demo

In [34]:
%%capture
import grpc 
import numpy as np
import hydro_serving_grpc as hs
import keras
from keras.datasets import mnist
from keras import backend as K
import os
from pathlib import Path

## Read data sample

In [35]:
path = Path(os.getcwd()).parent
Xtest = np.load(f'{path}/data/SHL5hip_300_X_test.npy')[:,0]
Ytest = np.load(f'{path}/data/SHL5hip_300_Y_test.npy')[:,0]
yval = np.array(Ytest)[1]
val = np.array(Xtest)[1].reshape(1,300,3,3).astype(np.float32)
val = val.flatten()

## Setup channel

In [36]:
channel = grpc.insecure_channel("localhost:9090") 
stub = hs.PredictionServiceStub(channel) 
model_spec = hs.ModelSpec(name="activity_recognition")

## Create tensors

In [37]:
tensor_shape = hs.TensorShapeProto(dim=[hs.TensorShapeProto.Dim(size=1), hs.TensorShapeProto.Dim(size=300),hs.TensorShapeProto.Dim(size=3),hs.TensorShapeProto.Dim(size=3)])
tensor = hs.TensorProto(dtype=hs.DT_DOUBLE, tensor_shape=tensor_shape, double_val=val)

## Send request to model servable

In [38]:
request = hs.PredictRequest(model_spec=model_spec, inputs={"x": tensor})
result = stub.Predict(request)
result = np.array(result.outputs['y'].double_val).argmax()
print(f'Result activity # {result}')
print(f'Actual activity # {yval.argmax()}')

Result activity # 2
Actual activity # 2
